<a href="https://colab.research.google.com/github/riiii-bit43/data-science/blob/main/Credit-card2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Credit card transactions - Project - 2.csv to Credit card transactions - Project - 2.csv


In [ ]:
import pandas as pd

# Load the uploaded file
df = pd.read_csv('Credit card transactions - Project - 2.csv')

# Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Display the first few rows
df.head()


/tmp/ipython-input-4-768273105.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


,index,City,Date,Card Type,Exp Type,Gender,Amount
0,0,"Delhi, India",2014-10-29,Gold,Bills,F,82475
1,1,"Greater Mumbai, India",2014-08-22,Platinum,Bills,F,32555
2,2,"Bengaluru, India",2014-08-27,Silver,Bills,F,101738
3,3,"Greater Mumbai, India",2014-04-12,Signature,Bills,F,123424
4,4,"Bengaluru, India",2015-05-05,Gold,Bills,F,171574


In [ ]:
# Total spend by city
city_spend = df.groupby('City')['Amount'].sum().sort_values(ascending=False)

# Top 5 cities
top5_cities = city_spend.head(5)

# Percentage contribution
total_spend = city_spend.sum()
percent_contribution = (top5_cities / total_spend) * 100

# Combine into a DataFrame
task1_result = pd.DataFrame({
    'Total Spend': top5_cities,
    '% Contribution': percent_contribution.round(2)
})

print(task1_result)


                       Total Spend  % Contribution
City                                              
Greater Mumbai, India    576751476           14.15
Bengaluru, India         572326739           14.05
Ahmedabad, India         567794310           13.93
Delhi, India             556929212           13.67
Kolkata, India           115466943            2.83


In [ ]:
# Create a 'Month' column in YYYY-MM format
df['Month'] = df['Date'].dt.to_period('M')

# Group by Card Type and Month to get monthly spend
monthly_spend = df.groupby(['Card Type', 'Month'])['Amount'].sum().reset_index()

# Find the month with the highest spend for each Card Type
idx = monthly_spend.groupby('Card Type')['Amount'].idxmax()
task2_result = monthly_spend.loc[idx].reset_index(drop=True)

print(task2_result)


   Card Type    Month    Amount
0       Gold  2015-01  55455064
1   Platinum  2014-08  57936507
2  Signature  2013-12  58799522
3     Silver  2015-03  59723549


In [ ]:
# Sort by Card Type and Date
df_sorted = df.sort_values(['Card Type', 'Date'])

# Calculate cumulative spend per Card Type
df_sorted['Cumulative Spend'] = df_sorted.groupby('Card Type')['Amount'].cumsum()

# Filter: first row where cumulative spend crosses ₹10,00,000 for each Card Type
task3_result = df_sorted[df_sorted['Cumulative Spend'] >= 1_000_000] \
                    .groupby('Card Type').first().reset_index()

# Show all columns as required
print(task3_result)


   Card Type  index              City       Date Exp Type Gender  Amount  \
0       Gold   1521      Delhi, India 2013-10-04     Food      M  281924   
1   Platinum    190  Ahmedabad, India 2013-10-05    Bills      F  612572   
2  Signature     72      Delhi, India 2013-10-04    Bills      F  550782   
3     Silver   7564  Bengaluru, India 2013-10-04     Food      F  205179   

     Month  Cumulative Spend  
0  2013-10           1272624  
1  2013-10           1537482  
2  2013-10           1285819  
3  2013-10           1115582  


In [ ]:
# Filter only Gold card transactions
gold_df = df[df['Card Type'].str.lower() == 'gold']

# Total spend per city for Gold card
city_spend_gold = gold_df.groupby('City')['Amount'].sum()

# Percentage contribution by each city
percent_contribution_gold = (city_spend_gold / city_spend_gold.sum()) * 100

# Find the city with the lowest % spend
lowest_city = percent_contribution_gold.idxmin()
lowest_value = percent_contribution_gold.min()

print(f"City with lowest % spend for Gold card: {lowest_city} ({lowest_value:.2f}%)")


City with lowest % spend for Gold card: Dhamtari, India (0.00%)


In [ ]:
# Group by City and Expense Type to get total spend
grouped = df.groupby(['City', 'Exp Type'])['Amount'].sum().reset_index()

# Function to extract highest and lowest expense types for each city
def get_high_low(group):
    highest = group.loc[group['Amount'].idxmax(), 'Exp Type']
    lowest = group.loc[group['Amount'].idxmin(), 'Exp Type']
    return pd.Series({'Highest_Expense_Type': highest, 'Lowest_Expense_Type': lowest})

# Apply the function groupwise
task5_result = grouped.groupby('City').apply(get_high_low).reset_index()

print(task5_result)


                 City Highest_Expense_Type Lowest_Expense_Type
0     Achalpur, India              Grocery       Entertainment
1     Adilabad, India                Bills                Food
2    Adityapur, India                 Food             Grocery
3        Adoni, India                Bills       Entertainment
4        Adoor, India                 Fuel               Bills
..                ...                  ...                 ...
981    Zaidpur, India                 Food               Bills
982    Zamania, India        Entertainment                Fuel
983       Zira, India        Entertainment             Grocery
984   Zirakpur, India                 Fuel       Entertainment
985  Zunheboto, India              Grocery       Entertainment

[986 rows x 3 columns]


/tmp/ipython-input-9-2009033136.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  task5_result = grouped.groupby('City').apply(get_high_low).reset_index()


In [ ]:
# Clean Gender values (strip and uppercase to be safe)
df['Gender'] = df['Gender'].str.strip().str.upper()

# Total spend per expense type (all genders)
total_spend = df.groupby('Exp Type')['Amount'].sum()

# Total spend by females ('F') per expense type
female_spend = df[df['Gender'] == 'F'].groupby('Exp Type')['Amount'].sum()

# Calculate % contribution
percent_female = (female_spend / total_spend) * 100

# Format results
task6_result = percent_female.reset_index().rename(columns={'Amount': '% Spend by Females'})
task6_result.columns = ['Exp Type', '% Spend by Females']
task6_result['% Spend by Females'] = task6_result['% Spend by Females'].fillna(0).round(2)

print(task6_result)

        Exp Type  % Spend by Females
0          Bills               63.95
1  Entertainment               49.37
2           Food               54.91
3           Fuel               49.71
4        Grocery               50.91
5         Travel               51.13


In [ ]:
# Make sure Month column exists
df['Month'] = df['Date'].dt.to_period('M')

# Total monthly spend per Card Type + Exp Type
monthly = df.groupby(['Card Type', 'Exp Type', 'Month'])['Amount'].sum().reset_index()

# Get Dec-2013 and Jan-2014 data
dec = monthly[monthly['Month'] == '2013-12']
jan = monthly[monthly['Month'] == '2014-01']

# Merge to compare
merged = pd.merge(jan, dec, on=['Card Type', 'Exp Type'], suffixes=('_jan', '_dec'))

# Calculate % growth
merged['MoM Growth %'] = ((merged['Amount_jan'] - merged['Amount_dec']) / merged['Amount_dec']) * 100

# Get the combo with the highest growth
task7_result = merged.sort_values('MoM Growth %', ascending=False).head(1)

print(task7_result[['Card Type', 'Exp Type', 'MoM Growth %']])


  Card Type Exp Type  MoM Growth %
5      Gold   Travel     87.920081


In [ ]:
# Extract day of week (Monday=0, Sunday=6)
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Filter for weekends (Saturday=5, Sunday=6)
weekend_df = df[df['DayOfWeek'].isin([5, 6])]

# Group by City and compute total spend and transaction count
weekend_stats = weekend_df.groupby('City').agg({
    'Amount': 'sum',
    'Date': 'count'  # using Date to count transactions
}).rename(columns={'Date': 'Transaction Count'})

# Compute ratio
weekend_stats['Spend per Transaction'] = weekend_stats['Amount'] / weekend_stats['Transaction Count']

# Find the city with the highest ratio
task8_result = weekend_stats.sort_values('Spend per Transaction', ascending=False).head(1)

print(task8_result)


                Amount  Transaction Count  Spend per Transaction
City                                                            
Sonepur, India  299905                  1               299905.0


In [ ]:
# Sort by City and Date
df_sorted = df.sort_values(['City', 'Date'])

# Add transaction number per city
df_sorted['Txn_No'] = df_sorted.groupby('City').cumcount() + 1

# Get only the rows where a city hits its 500th transaction
txn_500 = df_sorted[df_sorted['Txn_No'] == 500][['City', 'Date']]
txn_500 = txn_500.rename(columns={'Date': 'Date_500'})

# Get the first transaction date per city
first_txn = df_sorted.groupby('City')['Date'].min().reset_index().rename(columns={'Date': 'Date_First'})

# Merge both
merged = pd.merge(txn_500, first_txn, on='City')

# Calculate days taken to reach 500th transaction
merged['Days Taken'] = (merged['Date_500'] - merged['Date_First']).dt.days

# City with the minimum days
task9_result = merged.sort_values('Days Taken').head(1)

print(task9_result[['City', 'Days Taken']])


               City  Days Taken
1  Bengaluru, India          81
